# Necessary imports

In [1]:
from IPython.display import clear_output

In [2]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

# Scrapping tables from wiki

In [3]:
earth_analogs = pd.read_html('https://en.wikipedia.org/wiki/Earth_analog')
earth_analogs = earth_analogs[0]
print('Size:', earth_analogs.size)
earth_analogs = earth_analogs.rename(
    columns = {
      "Name": "Object",
      "Earth masses (ME)": "Mass (M⊕)",
      "Earth radii (R🜨)": "Radius (R⊕)"
      })
earth_analogs.head()

Size: 40


,Object,Mass (M⊕),Radius (R⊕),Note
0,Kepler-69c,≙2.14,1.7,Originally thought to be in the circumstellar ...
1,Kepler-9d,>1.5[27],1.64,Extremely hot.
2,CoRoT-7b,<9,1.58,Extremely hot.
3,Kepler-20f,< 14.3[25],1.03[25],"Slightly larger and likely more massive, far t..."
4,Tau Ceti g,>1.75,NaN,Extremely hot. Not known to transit.


In [4]:
import pandas as pd
import numpy as np

potentially = pd.read_html('https://en.wikipedia.org/wiki/List_of_potentially_habitable_exoplanets')
potentially = potentially[1]
print('Size:', potentially.size)
potentially = potentially.rename(
    columns = {
      "Refs/Notes": "Note"
      })
potentially.head()

Size: 726


,Object,Star,Star type,Mass (M⊕),Radius (R⊕),Density (g/cm3),Flux (F⊕),Teq (K),Period (days),Distance (ly),Refs/notes
0,Earth (reported for reference),Sun,G2V,1.00,1.00,5.514,1.00,255,365.25,0,Only planet confirmed to support life
1,Venus (reported for reference),Sun,G2V,0.815,0.950,5.243,1.911,244.261,224.70,0.0000042,[5]
2,Mars (reported for reference),Sun,G2V,0.107,0.533,3.934,0.431,209.8,686.98,0.0000058 - 0.000042,[6]
3,Gliese 12 b,Gliese 12,M4V,0.88+0.39 −0.26,1.03±0.11,4.44,1.6±0.2,315,12.70,40,Might resemble Venus and be unhabitable[7]
4,Gliese 163 c,Gliese 163,M3V,≥6.80,—,—,1.25,277,25.60,49,[1]


# Scrapping table from NASA

In [5]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import requests

def load_data(nasa_url, local_path):
    """Load data taking in the TAP protocol URL for NASA and the local path
    """
    request_csv = requests.get(nasa_url)
    with open(local_path, 'w') as f:
        f.write(request_csv.text)
    data = pd.read_csv(local_path)
    return data


def parsec_to_ly(data):
    """Rename columns and convert distance from parsec to light years
    """
    data['Distance (ly)'] = data['sy_dist'] * 3.26
    data = data.drop(columns = 'sy_dist')
    return data

In [6]:
# Some variables
nasa_url = "https://exoplanetarchive.ipac.caltech.edu/TAP/sync?query=select+pl_name,sy_dist,pl_rade,pl_masse,disc_year,discoverymethod+from+ps&format=csv"
local_path = "/content/exoplanet_data.csv"
n_clusters = 4
n_init = 40
kmeans_columns = ['Radius (R⊕)', 'Mass (M⊕)']

In [7]:
exoplanet_data = load_data(nasa_url, local_path)

In [8]:
exoplanet_data = exoplanet_data.rename(
    columns = {
      "pl_name": "Object",
      "pl_rade": "Radius (R⊕)",
      "pl_masse": "Mass (M⊕)",
      "disc_year": "Discovery Year",
      "discoverymethod": "Discovery Method"
      })

exoplanet_data = parsec_to_ly(exoplanet_data)
exoplanet_data = exoplanet_data.drop_duplicates(subset=['Object'])

In [9]:
print('Size:', exoplanet_data.size)

exoplanet_data.head()

Size: 35202


,Object,Radius (R⊕),Mass (M⊕),Discovery Year,Discovery Method,Distance (ly)
0,Kepler-6 b,13.38,NaN,2009,Transit,1913.74714
18,Kepler-491 b,10.00,NaN,2016,Transit,2056.35910
29,Kepler-257 b,2.74,NaN,2014,Transit,2543.63456
40,Kepler-216 b,2.12,NaN,2014,Transit,3871.15220
52,Kepler-32 c,2.37,NaN,2011,Transit,1055.74122


In [10]:
exoplanet_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5867 entries, 0 to 38168
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Object            5867 non-null   object 
 1   Radius (R⊕)       3491 non-null   float64
 2   Mass (M⊕)         1049 non-null   float64
 3   Discovery Year    5867 non-null   int64  
 4   Discovery Method  5867 non-null   object 
 5   Distance (ly)     5740 non-null   float64
dtypes: float64(3), int64(1), object(2)
memory usage: 320.9+ KB


In [11]:
print(list(exoplanet_data.columns))
print(list(earth_analogs.columns))
print(list(potentially.columns))

['Object', 'Radius (R⊕)', 'Mass (M⊕)', 'Discovery Year', 'Discovery Method', 'Distance (ly)']
['Object', 'Mass (M⊕)', 'Radius (R⊕)', 'Note']
['Object', 'Star', 'Star type', 'Mass (M⊕)', 'Radius (R⊕)', 'Density (g/cm3)', 'Flux (F⊕)', 'Teq (K)', 'Period (days)', 'Distance (ly)', 'Refs/notes']


In [12]:
exoplanet_data.isnull().sum()

,0
Object,0
Radius (R⊕),2376
Mass (M⊕),4818
Discovery Year,0
Discovery Method,0
Distance (ly),127


In [13]:
# Deleting data with unknown mass
exoplanet_data = exoplanet_data.dropna().reset_index()
exoplanet_data.size

3955

In [14]:
frames = [potentially, earth_analogs, exoplanet_data]

all_planets = pd.concat(frames)
print(list(all_planets.columns))
print('Size:', all_planets.size)
all_planets = all_planets.drop_duplicates(subset=['Object'])
all_planets.head()
# columns = ['Object', 'Star', 'Star type', 'Mass (M⊕)',	'Radius (R⊕)',	'Density (g/cm3)',	'Flux (F⊕)',	'Temperature (K)',	'Period (days)',	'Distance (ly)', "Discovery Year",  "Discovery Method"]


['Object', 'Star', 'Star type', 'Mass (M⊕)', 'Radius (R⊕)', 'Density (g/cm3)', 'Flux (F⊕)', 'Teq (K)', 'Period (days)', 'Distance (ly)', 'Refs/notes', 'Note', 'index', 'Discovery Year', 'Discovery Method']
Size: 9615


,Object,Star,Star type,Mass (M⊕),Radius (R⊕),Density (g/cm3),Flux (F⊕),Teq (K),Period (days),Distance (ly),Refs/notes,Note,index,Discovery Year,Discovery Method
0,Earth (reported for reference),Sun,G2V,1.00,1.00,5.514,1.00,255,365.25,0,Only planet confirmed to support life,NaN,NaN,NaN,NaN
1,Venus (reported for reference),Sun,G2V,0.815,0.950,5.243,1.911,244.261,224.70,0.0000042,[5],NaN,NaN,NaN,NaN
2,Mars (reported for reference),Sun,G2V,0.107,0.533,3.934,0.431,209.8,686.98,0.0000058 - 0.000042,[6],NaN,NaN,NaN,NaN
3,Gliese 12 b,Gliese 12,M4V,0.88+0.39 −0.26,1.03±0.11,4.44,1.6±0.2,315,12.70,40,Might resemble Venus and be unhabitable[7],NaN,NaN,NaN,NaN
4,Gliese 163 c,Gliese 163,M3V,≥6.80,—,—,1.25,277,25.60,49,[1],NaN,NaN,NaN,NaN


# Preprocess

In [15]:
columns = ['Object', 'Star', 'Star type', 'Mass (M⊕)', 'Radius (R⊕)', 'Density (g/cm3)', 'Flux (F⊕)', 'Teq (K)', 'Period (days)', 'Distance (ly)', 'Note', 'Discovery Year', 'Discovery Method']
format_of_columns = ['str', 'str', 'str', 'float', 'float', 'float', 'float', 'float', 'float', 'float', 'str', 'str', 'str']

bad_chars=['+', '-', '±', '—', '≥', '~', '[', ']']
all_planets.head(5)

,Object,Star,Star type,Mass (M⊕),Radius (R⊕),Density (g/cm3),Flux (F⊕),Teq (K),Period (days),Distance (ly),Refs/notes,Note,index,Discovery Year,Discovery Method
0,Earth (reported for reference),Sun,G2V,1.00,1.00,5.514,1.00,255,365.25,0,Only planet confirmed to support life,NaN,NaN,NaN,NaN
1,Venus (reported for reference),Sun,G2V,0.815,0.950,5.243,1.911,244.261,224.70,0.0000042,[5],NaN,NaN,NaN,NaN
2,Mars (reported for reference),Sun,G2V,0.107,0.533,3.934,0.431,209.8,686.98,0.0000058 - 0.000042,[6],NaN,NaN,NaN,NaN
3,Gliese 12 b,Gliese 12,M4V,0.88+0.39 −0.26,1.03±0.11,4.44,1.6±0.2,315,12.70,40,Might resemble Venus and be unhabitable[7],NaN,NaN,NaN,NaN
4,Gliese 163 c,Gliese 163,M3V,≥6.80,—,—,1.25,277,25.60,49,[1],NaN,NaN,NaN,NaN


In [16]:
import re

def preprocess_float(a):
  a=str(a)
  if a.find('+')!=-1:
    a=a[:a.find('+')]
  if a.find('-')!=-1:
    a=a[:a.find('-')]
  if a.find('±')!=-1:
    a=a[:a.find('±')]
  if a.find('≥')!=-1:
    a=a[a.find('≥')+1:]
  if a.find('>')!=-1:
    a=a[a.find('>')+1:]
  if a.find('<')!=-1:
    a=a[a.find('<')+1:]
  if a.find('≙')!=-1:
    a=a[a.find('≙')+1:]
  if a.find('~')!=-1:
    a=a[a.find('~')+1:]
  if a.find('—')!=-1:
    a='nan'
  if a.find('[')!=-1:
    a=a[:a.find('[')]
  cleaned_value = re.sub(r"[^0-9.-]", "", a)
  if cleaned_value =='':
    cleaned_value = 'nan'
  return round(float(cleaned_value.strip()), 2)

def preprocess_int(a):
  a=str(a)
  if a == 'NaN' or a=='nan' or a=='None':
    a='0'
  cleaned_value = re.sub(r"[^0-9.-]", "", a)
  return int(float(cleaned_value))

In [17]:
columns_float=['Mass (M⊕)', 'Radius (R⊕)', 'Density (g/cm3)', 'Flux (F⊕)', 'Teq (K)', 'Period (days)', 'Distance (ly)']
column_int=['Discovery Year']

In [18]:
for i in columns_float:
  all_planets[i] = all_planets[i].apply(lambda x: preprocess_float(x))

for i in column_int:
  all_planets[i] = all_planets[i].apply(lambda x: preprocess_int(x))
all_planets.head()

,Object,Star,Star type,Mass (M⊕),Radius (R⊕),Density (g/cm3),Flux (F⊕),Teq (K),Period (days),Distance (ly),Refs/notes,Note,index,Discovery Year,Discovery Method
0,Earth (reported for reference),Sun,G2V,1.00,1.00,5.51,1.00,255.00,365.25,0.0,Only planet confirmed to support life,NaN,NaN,0,NaN
1,Venus (reported for reference),Sun,G2V,0.81,0.95,5.24,1.91,244.26,224.70,0.0,[5],NaN,NaN,0,NaN
2,Mars (reported for reference),Sun,G2V,0.11,0.53,3.93,0.43,209.80,686.98,0.0,[6],NaN,NaN,0,NaN
3,Gliese 12 b,Gliese 12,M4V,0.88,1.03,4.44,1.60,315.00,12.70,40.0,Might resemble Venus and be unhabitable[7],NaN,NaN,0,NaN
4,Gliese 163 c,Gliese 163,M3V,6.80,NaN,NaN,1.25,277.00,25.60,49.0,[1],NaN,NaN,0,NaN


In [19]:
all_planets.info()

<class 'pandas.core.frame.DataFrame'>
Index: 636 entries, 0 to 564
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Object            636 non-null    object 
 1   Star              66 non-null     object 
 2   Star type         66 non-null     object 
 3   Mass (M⊕)         617 non-null    float64
 4   Radius (R⊕)       622 non-null    float64
 5   Density (g/cm3)   9 non-null      float64
 6   Flux (F⊕)         63 non-null     float64
 7   Teq (K)           55 non-null     float64
 8   Period (days)     66 non-null     float64
 9   Distance (ly)     628 non-null    float64
 10  Refs/notes        57 non-null     object 
 11  Note              8 non-null      object 
 12  index             562 non-null    float64
 13  Discovery Year    636 non-null    int64  
 14  Discovery Method  562 non-null    object 
dtypes: float64(8), int64(1), object(6)
memory usage: 79.5+ KB


In [20]:
all_planets.to_csv('DVW_project.csv', index=False)

# Filling with NASA data

In [21]:
columns_float=['Object', 'Star',	'Star type', 'Mass (M⊕)', 'Radius (R⊕)', 'Flux (F⊕)', 'Teq (K)', 'Period (days)', 'Distance (ly)', 'Note']
# all_planets[columns_float].dropna()
all_planets = all_planets[columns_float]
all_planets

,Object,Star,Star type,Mass (M⊕),Radius (R⊕),Flux (F⊕),Teq (K),Period (days),Distance (ly),Note
0,Earth (reported for reference),Sun,G2V,1.00,1.00,1.00,255.00,365.25,0.00,NaN
1,Venus (reported for reference),Sun,G2V,0.81,0.95,1.91,244.26,224.70,0.00,NaN
2,Mars (reported for reference),Sun,G2V,0.11,0.53,0.43,209.80,686.98,0.00,NaN
3,Gliese 12 b,Gliese 12,M4V,0.88,1.03,1.60,315.00,12.70,40.00,NaN
4,Gliese 163 c,Gliese 163,M3V,6.80,NaN,1.25,277.00,25.60,49.00,NaN
...,...,...,...,...,...,...,...,...,...,...
560,Kepler-197 c,NaN,NaN,5.30,1.23,NaN,NaN,NaN,1077.67,NaN
561,Kepler-100 c,NaN,NaN,7.05,2.20,NaN,NaN,NaN,993.14,NaN
562,Kepler-11 c,NaN,NaN,5.90,2.73,NaN,NaN,NaN,2107.09,NaN
563,Kepler-28 c,NaN,NaN,10.90,2.77,NaN,NaN,NaN,1429.05,NaN


In [22]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm  # Import tqdm for progress tracking


# Function to scrape data for a given object
def scrape_exoplanet_data(object_name):
    # Construct the URL dynamically
    url = f"https://science.nasa.gov/exoplanet-catalog/{object_name.lower().replace(' ', '-')}/"

    # Fetch the webpage content
    response = requests.get(url)
    if response.status_code != 200:
        return None

    # Parse the HTML content
    soup = BeautifulSoup(response.content, 'html.parser')

    try:
        # Object Name
        object_name_scraped = soup.find("h1").text.strip() if soup.find("h1") else 'nan'

        # Planet Radius
        radius_tag = soup.find("div", string="Planet Radius:")
        planet_radius = radius_tag.find_next("span").text.strip() if radius_tag and radius_tag.find_next("span") else 'nan'

        # Planet Mass
        mass_tag = soup.find("div", string="Planet Mass:")
        planet_mass = mass_tag.find_next("span").text.strip() if mass_tag and mass_tag.find_next("span") else 'nan'

        # Orbital Radius
        orbital_radius_tag = soup.find("div", string="Orbital Radius:")
        orbital_radius = orbital_radius_tag.find_next("span").text.strip() if orbital_radius_tag and orbital_radius_tag.find_next("span") else 'nan'

        # Orbital Period
        orbital_period_tag = soup.find("div", string="Orbital Period:")
        orbital_period = orbital_period_tag.find_next("span").text.strip() if orbital_period_tag and orbital_period_tag.find_next("span") else 'nan'

        # Note
        note_div = soup.find("div", class_="custom-field grid-col-12 padding-1")
        note_text = note_div.find_all("span")[1].text.strip() if note_div else "No additional information available"

    except Exception as e:
        return None

    # Return the scraped data as a dictionary
    return {
        "Object": object_name_scraped,
        "Mass (M⊕)": planet_mass,
        "Radius (R⊕)": planet_radius,
        "Period (days)": orbital_period,
        "Note": note_text
    }

skipped = []
# Add tqdm to track progress
for index, row in tqdm(all_planets.iterrows(), total=len(all_planets), desc="Scraping Progress"):
    if index>65:
        object_name = row["Object"].strip()

        # Skip rows where the Object name is missing
        if not object_name:
            continue

        # Scrape data for the current object
        scraped_data = scrape_exoplanet_data(object_name)
        if scraped_data is None:
            skipped.append(object_name)
            continue

        # Update missing values in the DataFrame
        for column in ["Mass (M⊕)", "Radius (R⊕)", "Period (days)"]:
            if pd.isna(row[column]) or row[column] == "nan":
                all_planets.at[index, column] = scraped_data[column]

        # Replace the Note column if the scraped note is not empty
        if scraped_data["Note"] != "No additional information available":
            all_planets.at[index, "Note"] = scraped_data["Note"]

clear_output()
# Save or display the updated DataFrame
all_planets.head()

,Object,Star,Star type,Mass (M⊕),Radius (R⊕),Flux (F⊕),Teq (K),Period (days),Distance (ly),Note
0,Earth (reported for reference),Sun,G2V,1.00,1.00,1.00,255.00,365.25,0.0,NaN
1,Venus (reported for reference),Sun,G2V,0.81,0.95,1.91,244.26,224.7,0.0,NaN
2,Mars (reported for reference),Sun,G2V,0.11,0.53,0.43,209.80,686.98,0.0,NaN
3,Gliese 12 b,Gliese 12,M4V,0.88,1.03,1.60,315.00,12.7,40.0,NaN
4,Gliese 163 c,Gliese 163,M3V,6.80,NaN,1.25,277.00,25.6,49.0,NaN


In [28]:
all_planets.to_csv('DVW_project.csv', index=False)

# LLM for fullfilling

In [23]:
!pip install -q google-generativeai
clear_output()

In [24]:
from google.colab import userdata
import google.generativeai as genai

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

In [27]:
model = genai.GenerativeModel('gemini-2.0-flash')
clear_output()

In [34]:
import pandas as pd
from tqdm import tqdm

# Define the columns for the DataFrame
columns = list(all_planets.columns)

# Initialize the DataFrame
ready_df = pd.DataFrame(columns=columns)

def generate_content(prompt: str) -> str:
    """
    Generate content using the model and return the response text.
    """
    try:
        response = model.generate_content(prompt)
        return response.text if hasattr(response, 'text') else "Sorry, I couldn't generate a response."
    except Exception as e:
        return f"There was an error generating the response: {str(e)}"


def generate_prompt_for_db(text):
    """
    Generate a prompt for preprocessing and filling in the table data.
    """
    prompt_input = f"""
You are an artificial intelligence assistant who corrects the table, preprocesses it, and fills in the blanks using NASA websites:
https://science.nasa.gov/exoplanets/exoplanet-catalog/
https://science.nasa.gov/exoplanets/target-star-catalog/
If the data is not found, fill it in another way. Ensure the number of columns remains consistent!

Columns of data:
{columns}
Format of columns: [str, str, str, float, float, float, float, float, float, str]

Input Data:
{text}

Output Format:
Provide the output data in this format without additional instructions or special characters (e.g., ±, —).
Example:
Earth (reported for reference)|Sun|G2V|1.00|1.00|1.00|255|365.25|0|This is our planet|
"""
    return generate_content(prompt_input)


def preprocess(row):
    """
    Preprocess a single row of data by generating and parsing the output.
    """
    # Convert the row to a pipe-separated string
    text = '|'.join(str(i) for i in row)

    # Generate the prompt and parse the output
    max_retries = 5
    for attempt in range(max_retries):
        out = generate_prompt_for_db(text).strip().split('|')
        if len(out) == len(columns):
            break
        print(f"Attempt {attempt + 1} failed for row {index}. Retrying...")
    else:
        raise ValueError(f"All retries failed for row {index}. Raw output: {'|'.join(out)}")

    # Validate the output length
    if len(out) != len(columns):
        raise ValueError(f"Unexpected number of columns in output: {len(out)}. Expected {len(columns)}. Raw output: {'|'.join(out)}")

    # Convert numeric columns to float
    for i in [3, 4, 5, 6, 7, 8]:  # Indices of numeric columns
        try:
            out[i] = float(out[i]) if out[i] else None  # Handle empty strings
        except ValueError:
            raise ValueError(f"Invalid float value in column {columns[i]}: {out[i]}")

    return out


# Process each row in the input DataFrame
for index, row in tqdm(all_planets.iterrows(), total=len(all_planets), desc="Processing Planets"):
    try:
        # Preprocess the row
        output = preprocess(list(row))

        # Add the processed row to the new DataFrame
        ready_df.loc[index] = output
    except Exception as e:
        print(f"Error processing row {index}: {e}")
        continue  # Skip rows with errors
clear_output()
# Display the final DataFrame
ready_df

In [32]:
ready_df.to_csv('planets_after_LLM.csv', index=False)

# Verifying ready-made data

In [36]:
df=pd.read_csv('/content/planets_after_LLM.csv')
df.head()

,Object,Star,Star type,Mass (M⊕),Radius (R⊕),Flux (F⊕),Teq (K),Period (days),Distance (ly),Note
0,Kepler-69c,Kepler-69,G4V,2.14,1.70,5.52,299.0,35.36,2700.0,Originally thought to be in the circumstellar ...
1,Kepler-9d,Kepler-9,G,1.50,1.64,244.19,1854.0,1.59,2350.0,Hot Neptune
2,Mars (reported for reference),Sun,G2V,0.11,0.53,0.43,210.0,687.00,0.0,Reference planet in Solar System
3,Gliese 180 d,Gliese 180,M2V,7.56,2.04,0.26,250.0,106.30,39.0,Confirmed exoplanet
4,Gliese 555 b,Gliese 555,M4V,5.46,1.77,0.50,214.0,36.20,20.4,Confirmed Planet


In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138 entries, 0 to 137
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Object         138 non-null    object 
 1   Star           138 non-null    object 
 2   Star type      138 non-null    object 
 3   Mass (M⊕)      137 non-null    float64
 4   Radius (R⊕)    138 non-null    float64
 5   Flux (F⊕)      136 non-null    float64
 6   Teq (K)        136 non-null    float64
 7   Period (days)  138 non-null    float64
 8   Distance (ly)  138 non-null    float64
 9   Note           138 non-null    object 
dtypes: float64(6), object(4)
memory usage: 10.9+ KB


In [38]:
df.describe()

,Mass (M⊕),Radius (R⊕),Flux (F⊕),Teq (K),Period (days),Distance (ly)
count,137.000000,138.000000,1.360000e+02,136.000000,1.380000e+02,138.000000
mean,207.590365,6.474855,1.589693e+04,1004.977941,1.642331e+04,884.619058
std,676.027936,8.230912,1.816735e+05,549.343938,1.781549e+05,1142.606257
min,0.110000,0.530000,0.000000e+00,0.000000,3.000000e-01,0.000000
25%,5.200000,1.747500,5.662500e+00,600.000000,3.500000e+00,178.560000
50%,12.200000,2.780000,8.535500e+01,972.500000,6.850000e+00,360.155000
75%,111.870000,11.257500,2.221375e+02,1425.000000,1.500000e+01,1273.877500
max,6261.220000,77.340000,2.118966e+06,2589.000000,2.090649e+06,6538.780000


In [39]:
df.isnull().sum()

,0
Object,0
Star,0
Star type,0
Mass (M⊕),1
Radius (R⊕),0
Flux (F⊕),2
Teq (K),2
Period (days),0
Distance (ly),0
Note,0


In [42]:
df = df.dropna()
df.isnull().sum()

,0
Object,0
Star,0
Star type,0
Mass (M⊕),0
Radius (R⊕),0
Flux (F⊕),0
Teq (K),0
Period (days),0
Distance (ly),0
Note,0


In [43]:
df.duplicated().sum()

np.int64(0)

In [44]:
df.to_csv('planets_after_LLM.csv', index=False)